In [ ]:
# import warnings
# warnings.filterwarnings('ignore')

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # загружаем open_ai_key

In [2]:
import json

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import (
SystemMessage, 
HumanMessage, 
AIMessage,
HumanMessagePromptTemplate, 
SystemMessagePromptTemplate
)
from langchain.chains import ConversationChain, LLMChain
from langchain.memory import ConversationBufferMemory, ChatMessageHistory

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.agents import OpenAIFunctionsAgent
from langchain.agents import AgentExecutor



from langchain.tools import DuckDuckGoSearchRun


In [4]:
model='gpt-3.5-turbo' # 4k токенов контекст
# model='gpt-3.5-turbo-16k' # 16k токенов контекст, подороже
# model='gpt-4' # дорогая для использования

In [5]:
llm = ChatOpenAI(temperature=0.0,model=model)
# chat

### Название

Никита делает

### Проблема

Создание промпта для переформулировки проблемы, которую присылает стартапер.

In [ ]:
role = '''Ты превосходный специалист по созданию питч деков. Сейчас ты помогаешь владельцу стартапа написать отличный питч дек.
'''

Затем мы описываем контекст того, как нужно расписать проблему и почему в соответствии с принципами Y Combinator

In [ ]:
details = '''
Первым этапом проработки питч-дека является проработка проблемы.
Проблема - это то, что пытается решить стартап. 
Проблема – это начальные условия, определяющие контекст, в котором ваша компания может расти быстро. Хорошая проблема популярна, растет, требует срочного решения, дорогостояща, обязательна и встречается часто. Инсайт в проблему показывает, почему ваше решение имеет конкурентное преимущество в плане роста. Выбирайте проблемы, которые соответствуют этим характеристикам, чтобы убедить инвесторов и привлечь внимание пользователей.
У проблемы, которая привлекательная для инвесторов есть характеристики:

1. Популярность: Популярность проблемы означает, что она актуальна для большого числа людей. Большой спрос на решение этой проблемы подразумевает, что существует значительная группа людей, которые сталкиваются с ней и ищут способы ее решения. Чем более широко проблема распространена, тем выше потенциал для быстрого роста вашей компании. В сущности, это уже текущий объем потенциальных клиентов.

2. Рост: Рост проблемы означает, что рынок, связанный с этой проблемой, растет быстрее, чем другие рынки. Это может быть обусловлено повышенным спросом на решение данной проблемы или изменениями в обществе, экономике или технологиях, которые стимулируют увеличение количества людей, сталкивающихся с этой проблемой. Иметь дело с растущей проблемой означает, что у вашей компании есть потенциал привлечь все больше пользователей и клиентов, что может способствовать быстрому росту бизнеса. В сущности, это степень роста рынка этой проблемы.

3. Срочность: Проблема считается срочной, если ее решение требуется немедленно или в ближайшем будущем. Срочные проблемы обычно вызывают дискомфорт, требуют внимания и действий в краткосрочной перспективе. Когда пользователи осознают, что проблема требует немедленного решения, это может способствовать активному поиску подходящих решений, включая продукты и услуги, предлагаемые вашим стартапом. В сущности, это характеристика или степень безотлагательности решения проблемы.

4. Затраты: В данном контексте затраты на решение проблемы скорее означают, что пользователи могут быть готовы платить значительные суммы за эффективное решение данной проблемы. То есть, если проблема дорогостояща для пользователей, то ваш стартап может предлагать решение, которое оправдывает эти затраты. В сущности, это денежный объем рынка проблемы.

5. Обязательность: Проблема с обязательностью означает, что люди должны решить ее по каким-то обстоятельствам. Это может быть связано с законодательством, регулированием, требованиями индустрии или другими факторами, которые обязывают пользователей найти способы решить данную проблему. Проблемы, которые обязательны к решению, создают своеобразную неотложность для пользователей, и это может стимулировать их активное взаимодействие с вашим стартапом, предоставляющим решение этой обязательной проблемы.

6. Частота: Проблема с частотой означает, что она проявляется у людей с высокой регулярностью или в определенные моменты времени. Это может быть проблема, с которой пользователи сталкиваются неоднократно, возможно, даже ежедневно или еженедельно. Проблемы с высокой частотой обеспечивают множество возможностей для вашего стартапа взаимодействовать с пользователями и предоставлять им решения. Когда проблема проявляется с высокой частотой, это может создать более постоянную потребность в решении, что способствует удержанию пользователей и стимулирует повторные взаимодействия с вашим продуктом или услугой.

Пример идеальных характеристик проблем для стартапа:
Миллионы пользователей сталкиваются с этой проблемой.
Рынок с проблемой растет на 20% ежегодно.
Проблему пытаются решить сразу.
Решение проблемы потребует больших затрат, что открывает возможности для высокой оплаты.
Законодательство изменилось, создавая дополнительные проблемы для решения.
Проблему необходимо решать несколько раз в день.
'''

И инструкцию, в которой просим чатгпт собрать необходимое кол-во информации, чтобы выяснить какова проблема  
в соответствии с деталями выше.
**Здесь нужно модифицировать инструкцию**:  
* Мы добавляем, что на выходе хотим получить содержание одного слайда.
* В этом слайде нужно описать ключевые моменты проблемы
* В создании содержания слайда мы в первую очередь опираемся на то, что говорит стартапер
* Модель, опираясь на слова стартапера, должна постараться написать эти ключевые моменты заинтересовать инвестора
* Характеристики проблемы, который интересны инвестору находятся в `details`
* Если по мнению модели 

Сначала модели нужно выяснить, достаточно ли стартапер предоставил информации, чтобы выяснить обладает ли их проблема характеристиками проблемы, привелекательной для клиента

В инструкции мы просим модель переписать текст стартапера так, чтобы выделить те характеристики, которые интересны инвесторам.

In [ ]:
instruction = '''
Задачи:
Постарайся расписать проблему, опираясь на текст стартапера, пройдясь по всем пунктам хорошей проблемы, которая заинтересовывает инвесторов.
Сформируй содержание слайда с акцентом на те характеристики, которые являются наиболее привлекательными для инвестора.
'''

In [ ]:
text_entry = '''
Текст стартапера:
{text}
'''

In [ ]:
conversation_prompt = role+details+instruction+text_entry

In [ ]:
# print(conversation_prompt)

Создаем шаблон.

In [ ]:
template = ChatPromptTemplate.from_template(conversation_prompt)

Создаем саму цепь, к которой будем обращаться в разговоре.

In [ ]:
# streaming_llm = ChatOpenAI(  
#  streaming=True,  
#  # callbacks=[StreamingStdOutCallbackHandler()],  
#  temperature=0,  
#  # openai_api_key=openai_api_key,
#  model='gpt-3.5-turbo-16k',
# );
# conversation = ConversationChain(
#     prompt=template,
#     llm=streaming_llm, 
#     memory=memory,
#     verbose=True
# )

In [ ]:
problem_chain = LLMChain(
    prompt=template,
    llm=llm, 
    verbose=True
)

Пример описания проблемы.

In [ ]:
test_description = '''У нас есть телеграм-бот, через который можно откликаться на разовые поручения или заявки на временную работу студенты регистрируются там и находятся в ожидании приходящих заявок мы заранее проговариваем с муниципальным предприятием условия гос.закупки на право заключение договора с обязательством предоставления в  течение 12 часов заявленного количества сотрудников, на некоторое количество раз в сезон в день ИКС уходят оповещения по нашей базе самозанятых о возможности выйти на двухдневную работу.
Для того, чтобы студент имел нужные знания, существуют стажировки. На обучение выпускника тратят определенную сумму денег и не факт, что он останется работать. Мы предлагаем, чтобы студент шел к нам, мы его обучали(давали узконаправленные знания для быстрого трудоустройства), после чего отправляем его на работу на определенное время, где он получает опыт. Спустя 3 месяца мы имем хорошего начального спкциалиста,  который спокойно сможет устроиться на работу. С каждой оплаты работу студента мы забираем 10 процентов себе. Мы планируем пойти дальше и получить лицензию на образовательную деятельность, чтобы иметь возможность разрабатывать свои обучающие программы, проработанные с экспертами предприятий (металлургическими комбинатами, операторами связи, заводами, инженерными компаниями), благодаря которым молодой специалист будет проходить "курс молодого бойца" исходя из реальных потребностей производственного сектора. Образование будет платным и укладывается в рамки ожиданий опрошенных нами респондентов.

Наши преимущества - мы сами студенты среднего профессионального учреждения, мы внутри проблемы и мы целевая аудитория.''' 

In [ ]:
test_answer = problem_chain(test_description)

In [ ]:
print(test_answer['text'])

Создадим корректирующую стиль цепь.

Промпт описывающий стиль.

In [ ]:
style_description = '''
СТИЛЬ:
```
Here are the detailed instructions or style of speech for each of the mentioned topics:

Simplicity and Clarity:

Use straightforward language that anyone can understand, avoiding complex terminology or convoluted explanations.
Present your idea in a way that leaves no room for interpretation or confusion. Make it crystal clear.
Focus on making your idea easily digestible, ensuring that even someone unfamiliar with your field can comprehend it.
Eliminate any vagueness or ambiguity in your description. Be explicit about what your startup does and the problem it solves.
Stay away from using jargon, buzzwords, or language that might not resonate with a broader audience.
Conciseness:

Condense your description to the most essential points. Trim away any extraneous details.
Craft a succinct and powerful statement that encapsulates the core of your startup's value proposition.
Prioritize brevity while still conveying the crucial aspects of your idea. Every word should serve a purpose.
Avoiding Padding:

Remove any unnecessary fluff or filler content from your pitch or description.
Stay focused on the key elements that make your startup unique and valuable. Discard information that doesn't contribute to this understanding.
Resist the temptation to add defensive explanations or preemptive responses to potential concerns. Let your idea stand on its merits.
"X for Y" Formula:

Choose a well-known and successful entity (X) that shares similarities with your startup's concept.
Clearly state the comparison: "We are the X for Y," indicating the familiar model you're adapting.
Highlight how Y, your target market or segment, can benefit from the attributes of X's model.
Ensure that the connection between X and Y is easily understandable, even to someone unfamiliar with either.
Empathy for Investors:

Put yourself in the shoes of investors who have limited time to review many ideas.
Craft your pitch and descriptions in a way that optimizes their efficiency and makes them enjoyable to read.
Present your startup idea in a format that quickly grabs their attention and conveys its unique value.
Conversational and Understandable:

Imagine explaining your idea to someone who is not well-versed in your industry or field.
Strive for a conversational tone, as if you were chatting with a friend about your startup.
Use language that resonates with a general audience, avoiding technical jargon or complex terminology.
Make your idea easy to remember and repeat. Aim for a description that naturally lends itself to word-of-mouth sharing.
Empathy for Users and Customers:

Prioritize the needs of your potential users and customers by making your idea accessible and comprehensible to them.
Frame your pitch in a way that resonates with a broad audience, not just those deeply involved in your industry.
Imagine someone who has never encountered your concept before and ensure that your description helps them grasp its significance.
In summary, the style of speech involves using simple, clear, and concise language while avoiding unnecessary padding or jargon. Incorporate empathy for both investors and potential users by crafting descriptions that are easy to understand, relatable, and compelling. Utilize the "X for Y" formula effectively to highlight your startup's value proposition.
```
'''

Промпт с инструкций о том, чтобы переформулировать входящий текст в соответствии со стилем.

In [ ]:
style_instruction = '''
ИНСТРУКЦИЯ:
Примени СТИЛЬ к ТЕКСТУ.

ТЕКСТ:
{text}
'''

In [ ]:
style_prompt = style_description + style_instruction

In [ ]:
style_template = ChatPromptTemplate.from_template(style_prompt)

In [ ]:
style_chain = LLMChain(
    prompt=style_template,
    llm=llm, 
    verbose=True
)

In [ ]:
test_style_answer = style_chain(test_answer['text'])

In [ ]:
print(test_style_answer['text'])

### Описание и ценностное предложение стартапа

Никита

### Решение

Промпт про unfair advantages by Y Comb, а именно супер продукт.  
Просим модель переформатировать текст для слайда и сделать акцент является ли продукт супер продуктом.

In [ ]:
super_product_description = '''
Супер Продукт:   
Очевидное преимущество - продукт, который значительно лучше конкурентов. Продукт должен быть на порядок (в 10 раз) более привлекательным и эффективным.
'''

In [ ]:
solution_instruction = '''
ЗАГЛУШКА
{text}
'''

In [ ]:
solution_prompt = super_product_description + solution_instruction

In [ ]:
solution_template = ChatPromptTemplate.from_template(solution_prompt)

In [ ]:
solution_chain = LLMChain(
    prompt=solution_template,
    llm=llm, 
    verbose=True
)

In [ ]:
test_solution_description = '''
'''

In [ ]:
test_solution_answer = solution_chain(test_solution_description)

In [ ]:
print(test_solution_answer['test'])

In [ ]:
style_description = '''
СТИЛЬ:
```
Here are the detailed instructions or style of speech for each of the mentioned topics:

Simplicity and Clarity:

Use straightforward language that anyone can understand, avoiding complex terminology or convoluted explanations.
Present your idea in a way that leaves no room for interpretation or confusion. Make it crystal clear.
Focus on making your idea easily digestible, ensuring that even someone unfamiliar with your field can comprehend it.
Eliminate any vagueness or ambiguity in your description. Be explicit about what your startup does and the problem it solves.
Stay away from using jargon, buzzwords, or language that might not resonate with a broader audience.
Conciseness:

Condense your description to the most essential points. Trim away any extraneous details.
Craft a succinct and powerful statement that encapsulates the core of your startup's value proposition.
Prioritize brevity while still conveying the crucial aspects of your idea. Every word should serve a purpose.
Avoiding Padding:

Remove any unnecessary fluff or filler content from your pitch or description.
Stay focused on the key elements that make your startup unique and valuable. Discard information that doesn't contribute to this understanding.
Resist the temptation to add defensive explanations or preemptive responses to potential concerns. Let your idea stand on its merits.
"X for Y" Formula:

Choose a well-known and successful entity (X) that shares similarities with your startup's concept.
Clearly state the comparison: "We are the X for Y," indicating the familiar model you're adapting.
Highlight how Y, your target market or segment, can benefit from the attributes of X's model.
Ensure that the connection between X and Y is easily understandable, even to someone unfamiliar with either.
Empathy for Investors:

Put yourself in the shoes of investors who have limited time to review many ideas.
Craft your pitch and descriptions in a way that optimizes their efficiency and makes them enjoyable to read.
Present your startup idea in a format that quickly grabs their attention and conveys its unique value.
Conversational and Understandable:

Imagine explaining your idea to someone who is not well-versed in your industry or field.
Strive for a conversational tone, as if you were chatting with a friend about your startup.
Use language that resonates with a general audience, avoiding technical jargon or complex terminology.
Make your idea easy to remember and repeat. Aim for a description that naturally lends itself to word-of-mouth sharing.
Empathy for Users and Customers:

Prioritize the needs of your potential users and customers by making your idea accessible and comprehensible to them.
Frame your pitch in a way that resonates with a broad audience, not just those deeply involved in your industry.
Imagine someone who has never encountered your concept before and ensure that your description helps them grasp its significance.
In summary, the style of speech involves using simple, clear, and concise language while avoiding unnecessary padding or jargon. Incorporate empathy for both investors and potential users by crafting descriptions that are easy to understand, relatable, and compelling. Utilize the "X for Y" formula effectively to highlight your startup's value proposition.
```
'''

In [ ]:
style_instruction = '''
ИНСТРУКЦИЯ:
Примени СТИЛЬ к ТЕКСТУ.

ТЕКСТ:
{text}
'''

In [ ]:
style_prompt = style_description + style_instruction


In [ ]:
style_template = ChatPromptTemplate.from_template(style_prompt)

In [ ]:
style_chain = LLMChain(
    prompt=style_template,
    llm=llm, 
    verbose=True
)

In [ ]:
test_solution_style_answer = style_chain(test_solution_answer['test'])

### Рынок

Тут пробовали сделать WebResearchRetrival для поиска первичной инфы для анализа TAM, SAM, SOM.  

Не заводится с DuckDuckGo, а гугл апи денег стоит.

In [ ]:
# import os  
# from langchain.vectorstores import Chroma  
# from langchain.embeddings import OpenAIEmbeddings  
# from langchain.chat_models.openai import ChatOpenAI  
# # from langchain.utilities import GoogleSearchAPIWrapper  
# from langchain.utilities import DuckDuckGoSearchAPIWrapper
# from langchain.retrievers.web_research import WebResearchRetriever  

# # Vectorstore  
# vectorstore = Chroma(embedding_function=OpenAIEmbeddings(),persist_directory="./chroma_db_oai")  
  
# # LLM  
# llm = ChatOpenAI(temperature=0)  
  
# # Search   
# search = DuckDuckGoSearchAPIWrapper()  

# # Initialize  
# web_research_retriever = WebResearchRetriever.from_llm(  
#  vectorstore=vectorstore,  
#  llm=llm,   
#  search=search,   
# )  

Попробуем сделать агента.

**OpenAIFunctionsAgent()** - works better.

In [6]:
# tools = [DuckDuckGoSearchRun()]

# system_message = SystemMessage(content="You are very powerful assistant, that creates a TOM, SAM, SOM analysis.")
# prompt = OpenAIFunctionsAgent.create_prompt(system_message=system_message)

# agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)

# agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# # agent_executor.run("Create a TAM, SAM, SOM analysis for a bicycle shop in russia exclusively.")


**AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION**

In [ ]:
# tools = [DuckDuckGoSearchRun()]
# agent = initialize_agent(
#     tools, llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True
# )
# agent.run("Create a TAM, SAM, SOM analysis for a bicycle shop in russia exclusively.")


План работы по слайду с рынком.
1. Дать стартаперу контекст о том, что такое TAM, SAM, SOM анализ.
2. Чтобы облегчить задачу, нужно найти информацию о TAM по сфере проблемы стартапера в деньгах.
3. Чтобы найти SAM минимум нужно воспользоваться формулой SAM = TAM * .1
4. А расчет SOM это задача стартапера, значит нужно дать контекст и помочь стартаперу рассчитать.
5. После сбора всей инфы, нужно из этого сделать слайд
6. И причесать в соответствии со стилем YComb

Чтобы сделать какой-то анализ нужно описание бизнеса и на какой рынок он рассчитывает (мировой или национальный).

* Описание бизнеса и географический тип рынка
  
По этому описанию находим TOM. С помощью агентов.

* Нужен агент, который находит в интернете TOM
  
Теперь нужно получить SAM, который мы получаем просто по формуле

* SAM = TOM * .1
* Или делаем агента, который пытается определить SAM с учетом TOM и специфика бизнеса
  
Теперь нужно получить SOM. Это задача стартапера, поэтому наша задача облегчить этот процесс. Каким образом?  

* Мы генерируем инструкцию о том, как сделать SOM.

#### Агент ищущий TOM

Здесь агент и саммаризатор.

In [28]:
tools = [DuckDuckGoSearchRun()]

In [29]:
system_message = SystemMessage(content="You are very powerful assistant, that creates a TOM, SAM, SOM analysis.")
prompt = OpenAIFunctionsAgent.create_prompt(system_message=system_message)

In [30]:
agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)

In [31]:
agent_executor_tom = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [32]:
example = 'Система для бронирования офисного пространства'
query = 'Производят катализаторы для нефтехимии'


In [33]:
tom_answer_1 = agent_executor_tom.run(f"Find Market Research for a {query} in russia. And give me a number either in rubbles or in amount of clients or demand.")



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `Market Research for Производят катализаторы для нефтехимии in Russia`


30/04/2021: Катализаторы для нефтехимии и нефтепереработки. Критическая зависимость от ... Потребности российских заводов в катализаторах гидропроцессов составляют, по разным оценкам, примерно 5-5,5 тыс. т в год. По состоянию на начало 2022 года наибольшую зависимость от импорта ... Например, катализаторы для каталитического крекинга — это микросферы диаметром около 70-90 микрометров, внешне похожие на мелкий песок. ... Где сейчас производят катализаторы для российских ... INTERFAX.RU - ООО "Газпромнефть-КС" проводит тендер на оказание услуг по проведению маркетингового исследования ... Его мощность составляет до 500 т в год. Завод выпускает катализаторы риформинга бензинов, изомеризации, а также катализаторы и адсорбенты для различных процессов нефтехимии.According to the search results, the market research for "Производят катализаторы дл

In [34]:
print(tom_answer_1)

According to the search results, the market research for "Производят катализаторы для нефтехимии" in Russia indicates that the annual demand for catalysts for hydroprocessing processes in Russian plants is estimated to be around 5,000-5,500 tons. As of the beginning of 2022, there is a significant dependence on imports for catalysts in Russia.

One example of a company producing catalysts for the Russian market is "ООО Газпромнефть-КС" (Gazpromneft-KS). Their plant has a capacity of up to 500 tons per year and produces catalysts for reforming gasoline, isomerization, as well as catalysts and adsorbents for various petrochemical processes.

Unfortunately, the search results do not provide specific information about the market size in terms of revenue or the number of clients.


Этот агент, кажется, работает лучше всего.  
Для него сделаем еще один запрос, который вытаскивает исключительно информацию об объеме рынка, кол-во клиентов или объеме спроса.

In [35]:
market_tom_summary_prompt = '''Extract only
market size:
number of clients:
demand: 

If there is no info write: na

Text to extract from:
{text}
'''

In [36]:
market_tom_summary_template = ChatPromptTemplate.from_template(market_tom_summary_prompt)

In [37]:
market_tom_summary_chain = LLMChain(
    prompt=market_tom_summary_template,
    llm=llm, 
    verbose=True
)

In [39]:
tom_summary = market_tom_summary_chain(tom_answer_1)



> Entering new LLMChain chain...
Prompt after formatting:
Human: Extract only
market size:
number of clients:
demand: 

If there is no info write: na

Text to extract from:
According to the search results, the market research for "Производят катализаторы для нефтехимии" in Russia indicates that the annual demand for catalysts for hydroprocessing processes in Russian plants is estimated to be around 5,000-5,500 tons. As of the beginning of 2022, there is a significant dependence on imports for catalysts in Russia.

One example of a company producing catalysts for the Russian market is "ООО Газпромнефть-КС" (Gazpromneft-KS). Their plant has a capacity of up to 500 tons per year and produces catalysts for reforming gasoline, isomerization, as well as catalysts and adsorbents for various petrochemical processes.

Unfortunately, the search results do not provide specific information about the market size in terms of revenue or the number of clients.


> Finished chain.


In [40]:
print(tom_summary['text'])

market size: na
number of clients: na
demand: 5,000-5,500 tons


#### SAM

##### Агент определяющий SAM сходу

In [20]:
tools = [DuckDuckGoSearchRun()]

In [21]:
system_message = SystemMessage(content="You are very powerful assistant, that creates a TOM, SAM, SOM analysis.")
prompt = OpenAIFunctionsAgent.create_prompt(system_message=system_message)

In [22]:
agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)

In [23]:
agent_executor_sam = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [46]:
example = 'Система для бронирования офисного пространства'
query = 'Производят катализаторы для нефтехимии'


In [47]:
description = '''"Компания ООО «РеоКат» работает в области сервиса катализаторов нефтепереработки и систем аминовой очистки газов. Компания специализируется на релизации проектов связанных с:
1. Мобильными комплексами высокоэффективной регенерации наноструктурированных катализаторов гидропроцессов нефтепереработки, включающий блок очистки отходящих газов.
2. Сервисе катализаторных систем: компания  оказывает услуги по выгрузке и загрузке катализаторов и адсорбентов, сульфидированию катализаторов гидроочистки.
3. Компания занимается очисткой аминовых растворов от механических примесей и термостабильных солей.Цеолит представляет собой гранулы, длиной 3-6 мм и диаметром 1-2 мм. При эксплуатации цеолита на установках очистки или осушки в течение нескольких лет на нем накапливаются сера и углерод, которые не могут быть удалены при регенерации в процессе и снижают его емкость. Цеолит выгружают из адсорберов и заменяют на новый. Отработанный цеолит направляется на утилизацию. Срок службы цеолита — от 6 месяцев до 4 лет."
'''

In [48]:
# prompt = f'''Company description: {description}

# Using this company description and this TOM info:
# {tom_answer_1}
# {tom_summary['text']}

# Estimate a percent of market or demand or clients that this company is aiming at and able to apprehend.
# Try to gather additional info about technology in this market in order to estimate the percent.

# '''

In [55]:
prompt = f'''Company description: {description}

Using this company description and this TOM info:
{tom_summary['text']}

Estimate a percent of market or demand or clients that this company is aiming at and able to apprehend.
'''

In [56]:
print(prompt)

Company description: "Компания ООО «РеоКат» работает в области сервиса катализаторов нефтепереработки и систем аминовой очистки газов. Компания специализируется на релизации проектов связанных с:
1. Мобильными комплексами высокоэффективной регенерации наноструктурированных катализаторов гидропроцессов нефтепереработки, включающий блок очистки отходящих газов.
2. Сервисе катализаторных систем: компания  оказывает услуги по выгрузке и загрузке катализаторов и адсорбентов, сульфидированию катализаторов гидроочистки.
3. Компания занимается очисткой аминовых растворов от механических примесей и термостабильных солей.Цеолит представляет собой гранулы, длиной 3-6 мм и диаметром 1-2 мм. При эксплуатации цеолита на установках очистки или осушки в течение нескольких лет на нем накапливаются сера и углерод, которые не могут быть удалены при регенерации в процессе и снижают его емкость. Цеолит выгружают из адсорберов и заменяют на новый. Отработанный цеолит направляется на утилизацию. Срок службы 

In [57]:
sam_answer = agent_executor_tom.run(prompt)



> Entering new AgentExecutor chain...
Based on the provided information, it is not possible to estimate the exact percentage of the market, demand, or clients that the company "ООО «РеоКат»" is aiming at and able to apprehend. The market size and number of clients are not specified, and the demand is given as a range of 5,000-5,500 tons without any context.

To make a more accurate estimation, additional information such as the total market size, the company's market share, and its competitive advantages would be needed. Without this information, it is not possible to determine the specific percentage of the market or demand that the company is targeting and capable of capturing.

> Finished chain.


In [45]:
print(sam_answer)

What would you like me to analyze?


Агентом не удается сходу сделать SAM.

##### Последовательный подход

**Попробуем другой подход**

1. Берем объем потребности в целом, который мы рассчитали в TAM.
2. Разделяем объем потребности по типам прямой и косвенной конкуренции.
3. Делаем допущение о доли рынка, которую мы хотим отхватить у конкурентов.
4. Делаем допущение о том, сколько новых клиентов мы смогли бы удовлетворить, которые ранее не пользовались ни одним из способов (например, потому что они были слишком дорогими).
5. Используя знания о количестве потребности на потребителя (получены при расчете TAM), рассчитываем количество клиентов и потребностейстей

###### Ищем прямых и косвенных конкурентов

In [58]:
# tools = [DuckDuckGoSearchRun()]
# system_message = SystemMessage(content="You are very powerful assistant, that creates a TOM, SAM, SOM analysis.")
# prompt = OpenAIFunctionsAgent.create_prompt(system_message=system_message)
# agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
# agent_executor_sam = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [72]:
description = '''"Компания ООО «РеоКат» работает в области сервиса катализаторов нефтепереработки и систем аминовой очистки газов. Компания специализируется на релизации проектов связанных с:
1. Мобильными комплексами высокоэффективной регенерации наноструктурированных катализаторов гидропроцессов нефтепереработки, включающий блок очистки отходящих газов.
2. Сервисе катализаторных систем: компания  оказывает услуги по выгрузке и загрузке катализаторов и адсорбентов, сульфидированию катализаторов гидроочистки.
3. Компания занимается очисткой аминовых растворов от механических примесей и термостабильных солей.Цеолит представляет собой гранулы, длиной 3-6 мм и диаметром 1-2 мм. При эксплуатации цеолита на установках очистки или осушки в течение нескольких лет на нем накапливаются сера и углерод, которые не могут быть удалены при регенерации в процессе и снижают его емкость. Цеолит выгружают из адсорберов и заменяют на новый. Отработанный цеолит направляется на утилизацию. Срок службы цеолита — от 6 месяцев до 4 лет."
'''


prompt = f'''Описание компании: {description}

Найди конкурентов в интернете:
* попробуй прямо искать конкурентов компании
* Если нет подходящей информации, попробуй найти все компании отрасли. Информацию об отрасли возьми из описания компании
* Найдя информацию о конкуренте, попробуй поискать конкурентов этого конкурента
'''

In [61]:
# sam_answer = agent_executor_sam.run(prompt)



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'arg1': 'конкуренты компании ООО РеоКат'}`




TypeError: DuckDuckGoSearchRun._run() got an unexpected keyword argument 'arg1'

In [73]:
tools = [DuckDuckGoSearchRun()]
agent = initialize_agent(
    tools, llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)
competitors = agent.run(prompt)



> Entering new AgentExecutor chain...
Thought: To find competitors of the company "ООО РеоКат", I will first search directly for competitors of the company. If I don't find any relevant information, I will search for companies in the same industry based on the description provided.

Action:
```
{
  "action": "duckduckgo_search",
  "action_input": "конкуренты ООО РеоКат"
}
```


Observation: Финансовое состояние ООО "РЕОКАТ" По сравнению с отраслевыми показателями в т.ч. изменение за 2021 год «РеоКат» примет участие в конференции «Минеральные удобрения 2023». Генеральный директор компании Генеральный директор компании Что можно узнать об организации ООО "РЕОКАТ" (Королёв, ИНН 9731049544) из ЕГРЮЛ и других официальных источников: директор, учредитель, выручка и много другое для проверки контрагента. Однако для компании РеоКат быть первым недостаточно. Для них было важно подчеркнуть, что они не только первые, но и единственные в России в своей сфере. ТОП-6 сервисов в 2023 году для анали

OutputParserException: Could not parse LLM output: 

Не получается пока сделать поиск по конкурентам. Поэтому откатимся к самому простому варианту, в котором проводим стартапера через самостоятальное определение SAM.

##### Самостоятельное определение SAM стартапером

Шаг 0: Берем TAM в виде денежного объема и/или кол-ва клиентов и/или объема спроса.

Шаг 1: Разделение Потребности по Типам Прямой и Косвенной Конкуренции

Идентифицируйте различные типы конкуренции на рынке - прямую (конкуренты, предоставляющие аналогичные продукты или услуги) и косвенную (альтернативные решения для удовлетворения потребности).
Разделите объем потребности из TAM между этими типами конкуренции, чтобы понять, какие части рынка контролируют ваши конкуренты, а какие - открыты для вас.

Шаг 2: Допущение о Доли Рынка

Проанализируйте, какую долю рынка вы хотели бы получить у конкурентов. Это может быть основано на вашей стратегии роста и конкурентных преимуществах.

Шаг 3: Допущение о Новых Клиентах

Рассмотрите возможность привлечения новых клиентов, которые ранее не пользовались аналогичными продуктами или услугами. Сделайте предположение о том, сколько из этих клиентов могли бы стать вашими клиентами благодаря вашим преимуществам (например, более низкая цена, улучшенные характеристики продукта).

Шаг 4: Рассчет Количества Клиентов и Потребностей

Используя знания о количестве потребности на потребителя (полученные при расчете TAM) и допущения из предыдущих шагов, рассчитайте количество клиентов и потребностей, которые вы можете охватить в рамках вашей стратегии.

Эту инструкцию кастомизируем под специфику бизнеса и ждем ввода инфы от стартапера.

Создадим цепь.

In [74]:
sam_instruction_for_client_prompt = '''

Измени шаблон инструкции в соответствии с описанием компании, чтобы владельцу компании было легко собрать необходимые данные и посчитать SAM.

Шаблон инструкции:
```
Шаг 0: Берем TAM в виде денежного объема и/или кол-ва клиентов и/или объема спроса.

Шаг 1: Разделение Потребности по Типам Прямой и Косвенной Конкуренции

Идентифицируйте различные типы конкуренции на рынке - прямую (конкуренты, предоставляющие аналогичные продукты или услуги) и косвенную (альтернативные решения для удовлетворения потребности).
Разделите объем потребности из TAM между этими типами конкуренции, чтобы понять, какие части рынка контролируют ваши конкуренты, а какие - открыты для вас.

Шаг 2: Допущение о Доли Рынка

Проанализируйте, какую долю рынка вы хотели бы получить у конкурентов. Это может быть основано на вашей стратегии роста и конкурентных преимуществах.

Шаг 3: Допущение о Новых Клиентах

Рассмотрите возможность привлечения новых клиентов, которые ранее не пользовались аналогичными продуктами или услугами. Сделайте предположение о том, сколько из этих клиентов могли бы стать вашими клиентами благодаря вашим преимуществам (например, более низкая цена, улучшенные характеристики продукта).

Шаг 4: Рассчет Количества Клиентов и Потребностей

Используя знания о количестве потребности на потребителя (полученные при расчете TAM) и допущения из предыдущих шагов, рассчитайте количество клиентов и потребностей, которые вы можете охватить в рамках вашей стратегии.
```

Описание компании:
{text}
'''

In [75]:
sam_instruction_for_client_template = ChatPromptTemplate.from_template(sam_instruction_for_client_prompt)

In [76]:
sam_instruction_for_client_chain = LLMChain(
    prompt=sam_instruction_for_client_template,
    llm=llm, 
    verbose=True
)

In [77]:
description = '''"Компания ООО «РеоКат» работает в области сервиса катализаторов нефтепереработки и систем аминовой очистки газов. Компания специализируется на релизации проектов связанных с:
1. Мобильными комплексами высокоэффективной регенерации наноструктурированных катализаторов гидропроцессов нефтепереработки, включающий блок очистки отходящих газов.
2. Сервисе катализаторных систем: компания  оказывает услуги по выгрузке и загрузке катализаторов и адсорбентов, сульфидированию катализаторов гидроочистки.
3. Компания занимается очисткой аминовых растворов от механических примесей и термостабильных солей.Цеолит представляет собой гранулы, длиной 3-6 мм и диаметром 1-2 мм. При эксплуатации цеолита на установках очистки или осушки в течение нескольких лет на нем накапливаются сера и углерод, которые не могут быть удалены при регенерации в процессе и снижают его емкость. Цеолит выгружают из адсорберов и заменяют на новый. Отработанный цеолит направляется на утилизацию. Срок службы цеолита — от 6 месяцев до 4 лет."
'''

In [78]:
sam_instruction = sam_instruction_for_client_chain(description)



> Entering new LLMChain chain...
Prompt after formatting:
Human: 

Измени шаблон инструкции в соответствии с описанием компании, чтобы владельцу компании было легко собрать необходимые данные и посчитать SAM.

Шаблон инструкции:
```
Шаг 0: Берем TAM в виде денежного объема и/или кол-ва клиентов и/или объема спроса.

Шаг 1: Разделение Потребности по Типам Прямой и Косвенной Конкуренции

Идентифицируйте различные типы конкуренции на рынке - прямую (конкуренты, предоставляющие аналогичные продукты или услуги) и косвенную (альтернативные решения для удовлетворения потребности).
Разделите объем потребности из TAM между этими типами конкуренции, чтобы понять, какие части рынка контролируют ваши конкуренты, а какие - открыты для вас.

Шаг 2: Допущение о Доли Рынка

Проанализируйте, какую долю рынка вы хотели бы получить у конкурентов. Это может быть основано на вашей стратегии роста и конкурентных преимуществах.

Шаг 3: Допущение о Новых Клиентах

Рассмотрите возможность привлечения новых к

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..



> Finished chain.


In [80]:
print(sam_instruction['text'])

Шаблон инструкции для компании ООО «РеоКат»:

Шаг 0: Берем TAM в виде денежного объема и/или кол-ва клиентов и/или объема спроса.

Определите общий рыночный потенциал для услуг компании ООО «РеоКат» в области сервиса катализаторов нефтепереработки и систем аминовой очистки газов. Это может быть выражено в денежном объеме, количестве клиентов или объеме спроса на такие услуги.

Шаг 1: Разделение Потребности по Типам Прямой и Косвенной Конкуренции

Идентифицируйте различные типы конкуренции на рынке - прямую (конкуренты, предоставляющие аналогичные услуги по сервису катализаторов и систем аминовой очистки) и косвенную (альтернативные решения для удовлетворения потребности, например, замена катализаторов или систем аминовой очистки). Разделите объем потребности из TAM между этими типами конкуренции, чтобы понять, какую часть рынка контролируют ваши конкуренты, а какую - открыта для вас.

Шаг 2: Допущение о Доли Рынка

Проанализируйте, какую долю рынка вы хотели бы получить у конкурентов в

#### SOM

Пример расчета SOM из статьи https://productframework.ru/tam_sam_som


1. Берем объем клиентов и потребностей из SAM
2. Определяем
    * Ресурсы необходимые для привлечения потребителей и обеспечения сделок
    * Последовательно определяем факторы ограничений, которые снижают потенциал компании на доступ к рассчитанному объему клиентов.
    * Пытаемся учесть изменчивость этих ресурсов за выбранный пероди оценки рынка
3. Делаем допущение о среднем чеке одной сделки
4. Рассчитываем итоговый объем клиентов и доходов.

**Детальная инструкция расчета SOM**

**Оценка Serviceable Obtainable Market (SOM)**

**Шаг 1: Берем объем клиентов и потребностей из SAM**
Изучите Serviceable Addressable Market (SAM), который вы предварительно определили. Это количество клиентов, которых вы можете реалистично обслужить, учитывая ваши ресурсы и ограничения.

**Шаг 2: Определяем**
1. Ресурсы необходимые для привлечения потребителей и обеспечения сделок. Оцените, сколько средств, персонала и времени потребуется для маркетинга, продаж и обслуживания клиентов.
2. Последовательно определяем факторы ограничений, которые снижают потенциал компании на доступ к рассчитанному объему клиентов. Это может включать в себя конкурентов, ограничения ресурсов и другие факторы, которые могут ограничить вашу способность привлечь клиентов.
3. Пытаемся учесть изменчивость этих ресурсов за выбранный период оценки рынка. Учтите, что ресурсы и условия могут меняться, поэтому стоит предусмотреть эту изменчивость в оценке.

**Шаг 3: Делаем допущение о среднем чеке одной сделки**
Оцените, какую сумму в среднем приносит одна сделка или клиент. Это может быть вашим средним доходом от продажи продукта или услуги.

**Шаг 4: Рассчитываем итоговый объем клиентов и доходов**
Используя данные из Шага 2, определите, сколько клиентов компания реально может обслуживать с учетом ресурсов и ограничений.
1. Умножьте количество клиентов, которых компания реально может обработать (из Шага 2) на ваше допущение о среднем чеке сделки. Это даст вам оценку доходов от тех клиентов, которых вы фактически сможете обслуживать.
2. Учтите факторы изменчивости и ограничения, которые вы установили на Шаге 2, чтобы сделать оценку более точной.


In [81]:
som_instruction_for_client_prompt = '''

Измени шаблон инструкции в соответствии с описанием компании, чтобы владельцу компании было легко собрать необходимые данные и посчитать SAM.

Шаблон инструкции:
```
**Шаг 1: Берем объем клиентов и потребностей из SAM**
Изучите Serviceable Addressable Market (SAM), который вы предварительно определили. Это количество клиентов, которых вы можете реалистично обслужить, учитывая ваши ресурсы и ограничения.

**Шаг 2: Определяем**
1. Ресурсы необходимые для привлечения потребителей и обеспечения сделок. Оцените, сколько средств, персонала и времени потребуется для маркетинга, продаж и обслуживания клиентов.
2. Последовательно определяем факторы ограничений, которые снижают потенциал компании на доступ к рассчитанному объему клиентов. Это может включать в себя конкурентов, ограничения ресурсов и другие факторы, которые могут ограничить вашу способность привлечь клиентов.
3. Пытаемся учесть изменчивость этих ресурсов за выбранный период оценки рынка. Учтите, что ресурсы и условия могут меняться, поэтому стоит предусмотреть эту изменчивость в оценке.

**Шаг 3: Делаем допущение о среднем чеке одной сделки**
Оцените, какую сумму в среднем приносит одна сделка или клиент. Это может быть вашим средним доходом от продажи продукта или услуги.

**Шаг 4: Рассчитываем итоговый объем клиентов и доходов**
Используя данные из Шага 2, определите, сколько клиентов компания реально может обслуживать с учетом ресурсов и ограничений.
1. Умножьте количество клиентов, которых компания реально может обработать (из Шага 2) на ваше допущение о среднем чеке сделки. Это даст вам оценку доходов от тех клиентов, которых вы фактически сможете обслуживать.
2. Учтите факторы изменчивости и ограничения, которые вы установили на Шаге 2, чтобы сделать оценку более точной.
```

Описание компании:
{text}
'''

In [82]:
som_instruction_for_client_template = ChatPromptTemplate.from_template(som_instruction_for_client_prompt)

In [83]:
som_instruction_for_client_chain = LLMChain(
    prompt=som_instruction_for_client_template,
    llm=llm, 
    verbose=True
)

In [84]:
description = '''"Компания ООО «РеоКат» работает в области сервиса катализаторов нефтепереработки и систем аминовой очистки газов. Компания специализируется на релизации проектов связанных с:
1. Мобильными комплексами высокоэффективной регенерации наноструктурированных катализаторов гидропроцессов нефтепереработки, включающий блок очистки отходящих газов.
2. Сервисе катализаторных систем: компания  оказывает услуги по выгрузке и загрузке катализаторов и адсорбентов, сульфидированию катализаторов гидроочистки.
3. Компания занимается очисткой аминовых растворов от механических примесей и термостабильных солей.Цеолит представляет собой гранулы, длиной 3-6 мм и диаметром 1-2 мм. При эксплуатации цеолита на установках очистки или осушки в течение нескольких лет на нем накапливаются сера и углерод, которые не могут быть удалены при регенерации в процессе и снижают его емкость. Цеолит выгружают из адсорберов и заменяют на новый. Отработанный цеолит направляется на утилизацию. Срок службы цеолита — от 6 месяцев до 4 лет."
'''

In [85]:
som_instruction = som_instruction_for_client_chain(description)



> Entering new LLMChain chain...
Prompt after formatting:
Human: 

Измени шаблон инструкции в соответствии с описанием компании, чтобы владельцу компании было легко собрать необходимые данные и посчитать SAM.

Шаблон инструкции:
```
**Шаг 1: Берем объем клиентов и потребностей из SAM**
Изучите Serviceable Addressable Market (SAM), который вы предварительно определили. Это количество клиентов, которых вы можете реалистично обслужить, учитывая ваши ресурсы и ограничения.

**Шаг 2: Определяем**
1. Ресурсы необходимые для привлечения потребителей и обеспечения сделок. Оцените, сколько средств, персонала и времени потребуется для маркетинга, продаж и обслуживания клиентов.
2. Последовательно определяем факторы ограничений, которые снижают потенциал компании на доступ к рассчитанному объему клиентов. Это может включать в себя конкурентов, ограничения ресурсов и другие факторы, которые могут ограничить вашу способность привлечь клиентов.
3. Пытаемся учесть изменчивость этих ресурсов за выбр

In [86]:
print(som_instruction['text'])

**Шаг 1: Берем объем клиентов и потребностей из SAM**
Изучите Serviceable Addressable Market (SAM), который вы предварительно определили. Это количество клиентов, которых вы можете реалистично обслужить, учитывая ваши ресурсы и ограничения.

**Шаг 2: Определяем**
1. Ресурсы необходимые для привлечения потребителей и обеспечения сделок. Оцените, сколько средств, персонала и времени потребуется для маркетинга, продаж и обслуживания клиентов в области сервиса катализаторов нефтепереработки и систем аминовой очистки газов.
2. Последовательно определяем факторы ограничений, которые снижают потенциал компании на доступ к рассчитанному объему клиентов в данной области. Это может включать в себя конкурентов, ограничения ресурсов и другие факторы, которые могут ограничить вашу способность привлечь клиентов в данной области.
3. Пытаемся учесть изменчивость этих ресурсов за выбранный период оценки рынка в области сервиса катализаторов нефтепереработки и систем аминовой очистки газов. Учтите, что 

### Конкуренты

Нужно найти конкурентов и постараться определить свои конкурентные преимущества.

In [81]:
prompt = '''
Описание компании:
{text}
'''

In [82]:
template = ChatPromptTemplate.from_template(prompt)

In [83]:
chain = LLMChain(
    prompt=template,
    llm=llm, 
    verbose=True
)

In [84]:
description = '''"Компания ООО «РеоКат» работает в области сервиса катализаторов нефтепереработки и систем аминовой очистки газов. Компания специализируется на релизации проектов связанных с:
1. Мобильными комплексами высокоэффективной регенерации наноструктурированных катализаторов гидропроцессов нефтепереработки, включающий блок очистки отходящих газов.
2. Сервисе катализаторных систем: компания  оказывает услуги по выгрузке и загрузке катализаторов и адсорбентов, сульфидированию катализаторов гидроочистки.
3. Компания занимается очисткой аминовых растворов от механических примесей и термостабильных солей.Цеолит представляет собой гранулы, длиной 3-6 мм и диаметром 1-2 мм. При эксплуатации цеолита на установках очистки или осушки в течение нескольких лет на нем накапливаются сера и углерод, которые не могут быть удалены при регенерации в процессе и снижают его емкость. Цеолит выгружают из адсорберов и заменяют на новый. Отработанный цеолит направляется на утилизацию. Срок службы цеолита — от 6 месяцев до 4 лет."
'''

In [85]:
answer = chain(description)



> Entering new LLMChain chain...
Prompt after formatting:
Human: 

Измени шаблон инструкции в соответствии с описанием компании, чтобы владельцу компании было легко собрать необходимые данные и посчитать SAM.

Шаблон инструкции:
```
**Шаг 1: Берем объем клиентов и потребностей из SAM**
Изучите Serviceable Addressable Market (SAM), который вы предварительно определили. Это количество клиентов, которых вы можете реалистично обслужить, учитывая ваши ресурсы и ограничения.

**Шаг 2: Определяем**
1. Ресурсы необходимые для привлечения потребителей и обеспечения сделок. Оцените, сколько средств, персонала и времени потребуется для маркетинга, продаж и обслуживания клиентов.
2. Последовательно определяем факторы ограничений, которые снижают потенциал компании на доступ к рассчитанному объему клиентов. Это может включать в себя конкурентов, ограничения ресурсов и другие факторы, которые могут ограничить вашу способность привлечь клиентов.
3. Пытаемся учесть изменчивость этих ресурсов за выбр

Сделаем с помощью агента с DuckDuckGo.

In [110]:
agent_description = '''You are very powerful assistant, that finds competitors in the internet. If your search doesnt yeild result,
you can be creative and try different creative search queries for up to 3 times.'''

In [111]:
tools = [DuckDuckGoSearchRun()]
system_message = SystemMessage(content=agent_description)
prompt = OpenAIFunctionsAgent.create_prompt(system_message=system_message)

llm_high_temp = ChatOpenAI(temperature=0.9,model=model)

agent = OpenAIFunctionsAgent(llm=llm_high_temp, tools=tools, prompt=prompt)
agent_executor_competitor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [112]:
query = 'Производят катализаторы для нефтехимии'
answer_competitors_list = agent_executor_competitor(f'Find competitors for a company that {query} in russia')



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `competitors of Производят катализаторы для нефтехимии in Russia`


30/04/2021: Катализаторы для нефтехимии и нефтепереработки. Критическая зависимость от ... 07:38, 24.03.2023 Российская нефтепереработка преодолевает санкции за счет развития собственных катализаторных производств Фото: catalysts.gazprom-neft.ru Год прошел с момента, как Евросоюз решил сделать "невозможной модернизацию российских нефтеперерабатывающих заводов" (НПЗ). Современные катализаторы нужны для выпуска всех видов топлива экологического стандарта Евро-5. Этот стандарт действует в Евросоюзе с 2008-2009 годов. Он позволяет превращать тяжелые нефтяные фракции в компоненты моторных топлив и сырье для нефтехимии. Суммарный спрос на эту группу катализаторов в России составляет порядка 12-12,5 тыс. т в ... 2022 Для производства из нефти топлив и более сложных продуктов, например полиэтилена или полипропилена, необходимы катализаторы. Многие из н

In [113]:
print(answer_competitors_list['output'])

Based on the search results, here are some potential competitors for a company that produces catalysts for petrochemicals in Russia:

1. Sibur: Sibur is one of the world's fastest-growing petrochemical players and Russia's leading polymer and rubber manufacturer. They are involved in various sectors of the petrochemical industry, including catalysts.

2. Albemarle Corporation: Albemarle Corporation is a global specialty chemicals company that provides catalysts and other innovative solutions for various industries, including the petrochemical sector.

3. W R Grace: W R Grace is a leading supplier of catalysts and other chemical products. They offer a wide range of catalyst solutions for the petrochemical industry.

4. BASF: BASF is a multinational chemical company that offers catalysts for various applications, including petrochemicals. They are known for their expertise in developing innovative and sustainable catalyst technologies.

5. Haldor Topsoe: Haldor Topsoe is a Danish company

**Генерация конкурентных преимущств**

In [106]:
agent_description = '''You are a very powerful assistant who determines the competitive advantages of the company in relation to its competitors. You are given the name of the company and its description, as well as the names of competitors. If necessary, you can search for additional information about competitors in order to deduce the company's competitive advantages.'''


In [114]:
tools = [DuckDuckGoSearchRun()]
system_message = SystemMessage(content=agent_description)
prompt = OpenAIFunctionsAgent.create_prompt(system_message=system_message)

llm_high_temp = ChatOpenAI(temperature=0.9,model=model)

agent = OpenAIFunctionsAgent(llm=llm_high_temp, tools=tools, prompt=prompt)
agent_executor_competitor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [116]:
# query = 'Производят катализаторы для нефтехимии'

description = '''"Компания ООО «РеоКат» работает в области сервиса катализаторов нефтепереработки и систем аминовой очистки газов. Компания специализируется на релизации проектов связанных с:
1. Мобильными комплексами высокоэффективной регенерации наноструктурированных катализаторов гидропроцессов нефтепереработки, включающий блок очистки отходящих газов.
2. Сервисе катализаторных систем: компания  оказывает услуги по выгрузке и загрузке катализаторов и адсорбентов, сульфидированию катализаторов гидроочистки.
3. Компания занимается очисткой аминовых растворов от механических примесей и термостабильных солей.Цеолит представляет собой гранулы, длиной 3-6 мм и диаметром 1-2 мм. При эксплуатации цеолита на установках очистки или осушки в течение нескольких лет на нем накапливаются сера и углерод, которые не могут быть удалены при регенерации в процессе и снижают его емкость. Цеолит выгружают из адсорберов и заменяют на новый. Отработанный цеолит направляется на утилизацию. Срок службы цеолита — от 6 месяцев до 4 лет."
'''

prompt = f'''Название и описание компании: {description}

Текст, в котором присутствуют конкуренты:
{answer_competitors_list['output']}

Тебе нужно вывести конкурентные преимущества компании.
'''

answer = agent_executor_competitor(prompt)



> Entering new AgentExecutor chain...
Компания ООО «РеоКат» имеет ряд конкурентных преимуществ, которые могут помочь ей выделиться среди своих конкурентов:

1. Специализация в области сервиса катализаторов нефтепереработки и систем аминовой очистки газов: Компания специализируется исключительно на этой области, что позволяет ей обладать глубокими знаниями и опытом в этих процессах.

2. Мобильные комплексы высокоэффективной регенерации наноструктурированных катализаторов гидропроцессов нефтепереработки: Компания предлагает инновационные и эффективные решения для регенерации катализаторов, что может сократить затраты на их замену и повысить производительность процессов нефтепереработки.

3. Знание и опыт в сфере сульфидирования катализаторов гидроочистки: Компания предоставляет услуги по сульфидированию катализаторов гидроочистки, что может помочь клиентам улучшить эффективность и долговечность своих систем очистки.

4. Очистка аминовых растворов от механических примесей и термостабиль

### Бизнес-модель и ценообразование 

Мы задаем вопрос стартаперу о бизнес-модели.

Напишите способ генерации дохода и основные затраты вашего бизнеса?  
А чатГПТ сперва выдает пример того, как он видит способ генерации дохода и основные затраты бизнеса.

ЧатГПТ отвечает самостоятельно на часть вопросов:
1. Какие каналы вы используете для привлечения клиентов?
2. Как вы видите идите масштабирование своей бизнес-модели?

Процедура:  
1. ЧатГПТ генерирует самостоятельно ответ на вопрос о способе генерации дохода и основных затратах бизнеса по описанию клиента
2. Мы предлагаем это описание в качестве примера и задаем этот вопрос стартаперу
3. ЧатГПТ сам отвечает на вопрос о каналах для привлечения клиентов
4. Предлагает этот пример стартаперу и просит ввести его самого
5. ЧатГПТ сам отвечает на вопрос о масштабировании бизнес модели
6. Предложить пример стартаперу и попросить ответить на вопрос

##### ЧатГПТ отвечает: Напишите способ генерации дохода и основные затраты вашего бизнеса?

In [118]:
prompt = '''Напиши способы генерация дохода и основные затраты бизнеса по описанию компании.

Описание компании:
{text}
'''

In [119]:
template = ChatPromptTemplate.from_template(prompt)

In [120]:
chain = LLMChain(
    prompt=template,
    llm=llm, 
    verbose=True
)

In [121]:
description = '''"Компания ООО «РеоКат» работает в области сервиса катализаторов нефтепереработки и систем аминовой очистки газов. Компания специализируется на релизации проектов связанных с:
1. Мобильными комплексами высокоэффективной регенерации наноструктурированных катализаторов гидропроцессов нефтепереработки, включающий блок очистки отходящих газов.
2. Сервисе катализаторных систем: компания  оказывает услуги по выгрузке и загрузке катализаторов и адсорбентов, сульфидированию катализаторов гидроочистки.
3. Компания занимается очисткой аминовых растворов от механических примесей и термостабильных солей.Цеолит представляет собой гранулы, длиной 3-6 мм и диаметром 1-2 мм. При эксплуатации цеолита на установках очистки или осушки в течение нескольких лет на нем накапливаются сера и углерод, которые не могут быть удалены при регенерации в процессе и снижают его емкость. Цеолит выгружают из адсорберов и заменяют на новый. Отработанный цеолит направляется на утилизацию. Срок службы цеолита — от 6 месяцев до 4 лет."
'''

In [122]:
answer = chain(description)



> Entering new LLMChain chain...
Prompt after formatting:
Human: Напиши способы генерация дохода и основные затраты бизнеса по описанию компании.

Описание компании:
"Компания ООО «РеоКат» работает в области сервиса катализаторов нефтепереработки и систем аминовой очистки газов. Компания специализируется на релизации проектов связанных с:
1. Мобильными комплексами высокоэффективной регенерации наноструктурированных катализаторов гидропроцессов нефтепереработки, включающий блок очистки отходящих газов.
2. Сервисе катализаторных систем: компания  оказывает услуги по выгрузке и загрузке катализаторов и адсорбентов, сульфидированию катализаторов гидроочистки.
3. Компания занимается очисткой аминовых растворов от механических примесей и термостабильных солей.Цеолит представляет собой гранулы, длиной 3-6 мм и диаметром 1-2 мм. При эксплуатации цеолита на установках очистки или осушки в течение нескольких лет на нем накапливаются сера и углерод, которые не могут быть удалены при регенерации

In [124]:
print(answer['text'])

Способы генерации дохода компании ООО «РеоКат»:
1. Продажа мобильных комплексов высокоэффективной регенерации наноструктурированных катализаторов гидропроцессов нефтепереработки и блоков очистки отходящих газов.
2. Предоставление услуг по выгрузке и загрузке катализаторов и адсорбентов, сульфидированию катализаторов гидроочистки.
3. Оказание услуг по очистке аминовых растворов от механических примесей и термостабильных солей.

Основные затраты бизнеса компании ООО «РеоКат»:
1. Затраты на закупку и производство мобильных комплексов и блоков очистки.
2. Затраты на обслуживание и ремонт оборудования.
3. Затраты на транспортировку и доставку оборудования и материалов.
4. Затраты на оплату труда сотрудников, включая специалистов по сервису и очистке.
5. Затраты на маркетинг и рекламу для привлечения клиентов.
6. Затраты на утилизацию отработанного цеолита и закупку нового цеолита для замены.
7. Затраты на аренду или покупку помещений для производства и офисных помещений.
8. Затраты на обуче

##### ЧатГПТ отвечает: Какие каналы вы используете для привлечения клиентов?

In [125]:
prompt = '''Напиши о том, какие каналы привлечения клиентов используются в бизнесе по описанию компании.

Описание компании:
{text}
'''

In [126]:
template = ChatPromptTemplate.from_template(prompt)

In [127]:
chain = LLMChain(
    prompt=template,
    llm=llm, 
    verbose=True
)

In [128]:
description = '''"Компания ООО «РеоКат» работает в области сервиса катализаторов нефтепереработки и систем аминовой очистки газов. Компания специализируется на релизации проектов связанных с:
1. Мобильными комплексами высокоэффективной регенерации наноструктурированных катализаторов гидропроцессов нефтепереработки, включающий блок очистки отходящих газов.
2. Сервисе катализаторных систем: компания  оказывает услуги по выгрузке и загрузке катализаторов и адсорбентов, сульфидированию катализаторов гидроочистки.
3. Компания занимается очисткой аминовых растворов от механических примесей и термостабильных солей.Цеолит представляет собой гранулы, длиной 3-6 мм и диаметром 1-2 мм. При эксплуатации цеолита на установках очистки или осушки в течение нескольких лет на нем накапливаются сера и углерод, которые не могут быть удалены при регенерации в процессе и снижают его емкость. Цеолит выгружают из адсорберов и заменяют на новый. Отработанный цеолит направляется на утилизацию. Срок службы цеолита — от 6 месяцев до 4 лет."
'''

In [129]:
answer = chain(description)



> Entering new LLMChain chain...
Prompt after formatting:
Human: Напиши о том, какие каналы привлечения клиентов используются в бизнесе по описанию компании.

Описание компании:
"Компания ООО «РеоКат» работает в области сервиса катализаторов нефтепереработки и систем аминовой очистки газов. Компания специализируется на релизации проектов связанных с:
1. Мобильными комплексами высокоэффективной регенерации наноструктурированных катализаторов гидропроцессов нефтепереработки, включающий блок очистки отходящих газов.
2. Сервисе катализаторных систем: компания  оказывает услуги по выгрузке и загрузке катализаторов и адсорбентов, сульфидированию катализаторов гидроочистки.
3. Компания занимается очисткой аминовых растворов от механических примесей и термостабильных солей.Цеолит представляет собой гранулы, длиной 3-6 мм и диаметром 1-2 мм. При эксплуатации цеолита на установках очистки или осушки в течение нескольких лет на нем накапливаются сера и углерод, которые не могут быть удалены при

In [130]:
print(answer['text'])

Для привлечения клиентов компания ООО «РеоКат» может использовать следующие каналы:

1. Прямые продажи: Компания может активно искать потенциальных клиентов и предлагать им свои услуги напрямую. Это может включать в себя отправку персонализированных предложений по электронной почте, проведение презентаций и встреч с потенциальными клиентами, а также активное участие в выставках и конференциях, связанных с нефтепереработкой и аминовой очисткой газов.

2. Рекомендации и отзывы клиентов: Компания может стремиться получить положительные отзывы от своих клиентов и использовать их в качестве рекомендаций для привлечения новых клиентов. Это может быть достигнуто через удовлетворение клиентов высоким качеством обслуживания и предоставляемых услуг, а также активное сбор и использование отзывов клиентов.

3. Онлайн-присутствие: Компания может использовать свой веб-сайт и социальные медиа для привлечения клиентов. На веб-сайте компании должна быть информация о предоставляемых услугах, преимуществ

##### ЧатГПТ отвечает: Как вы видите идите масштабирование своей бизнес-модели?

In [131]:
prompt = '''Напиши о том, как можно масштабировать бизнес модель по описанию компании.

Описание компании:
{text}
'''

In [132]:
template = ChatPromptTemplate.from_template(prompt)

In [133]:
chain = LLMChain(
    prompt=template,
    llm=llm, 
    verbose=True
)

In [134]:
description = '''"Компания ООО «РеоКат» работает в области сервиса катализаторов нефтепереработки и систем аминовой очистки газов. Компания специализируется на релизации проектов связанных с:
1. Мобильными комплексами высокоэффективной регенерации наноструктурированных катализаторов гидропроцессов нефтепереработки, включающий блок очистки отходящих газов.
2. Сервисе катализаторных систем: компания  оказывает услуги по выгрузке и загрузке катализаторов и адсорбентов, сульфидированию катализаторов гидроочистки.
3. Компания занимается очисткой аминовых растворов от механических примесей и термостабильных солей.Цеолит представляет собой гранулы, длиной 3-6 мм и диаметром 1-2 мм. При эксплуатации цеолита на установках очистки или осушки в течение нескольких лет на нем накапливаются сера и углерод, которые не могут быть удалены при регенерации в процессе и снижают его емкость. Цеолит выгружают из адсорберов и заменяют на новый. Отработанный цеолит направляется на утилизацию. Срок службы цеолита — от 6 месяцев до 4 лет."
'''

In [135]:
answer = chain(description)



> Entering new LLMChain chain...
Prompt after formatting:
Human: Напиши о том, как можно масштабировать бизнес модель по описанию компании.

Описание компании:
"Компания ООО «РеоКат» работает в области сервиса катализаторов нефтепереработки и систем аминовой очистки газов. Компания специализируется на релизации проектов связанных с:
1. Мобильными комплексами высокоэффективной регенерации наноструктурированных катализаторов гидропроцессов нефтепереработки, включающий блок очистки отходящих газов.
2. Сервисе катализаторных систем: компания  оказывает услуги по выгрузке и загрузке катализаторов и адсорбентов, сульфидированию катализаторов гидроочистки.
3. Компания занимается очисткой аминовых растворов от механических примесей и термостабильных солей.Цеолит представляет собой гранулы, длиной 3-6 мм и диаметром 1-2 мм. При эксплуатации цеолита на установках очистки или осушки в течение нескольких лет на нем накапливаются сера и углерод, которые не могут быть удалены при регенерации в про

In [136]:
print(answer['text'])

Для масштабирования бизнес модели компании ООО «РеоКат» в области сервиса катализаторов нефтепереработки и систем аминовой очистки газов можно использовать следующие подходы:

1. Расширение географического охвата: Компания может начать предоставлять свои услуги в других регионах или странах. Это позволит привлечь новых клиентов и увеличить объемы продаж.

2. Развитие новых продуктов и услуг: Компания может исследовать и разработать новые продукты и услуги, связанные с областью сервиса катализаторов и систем аминовой очистки газов. Например, можно разработать новые методы регенерации катализаторов или новые технологии очистки аминовых растворов.

3. Партнерство с другими компаниями: Компания может заключить партнерские соглашения с другими компаниями, которые работают в смежных областях. Например, можно сотрудничать с производителями нефтеперерабатывающего оборудования или с поставщиками аминовых растворов.

4. Автоматизация и оптимизация процессов: Компания может внедрить новые техноло

### Трекшн и финансы

Нужно чтобы стартапер ввел годовую выручку и кол-во клиентов.

Тогда можно рассчитать: 
APRU
Churn Rate 
LT
LTV

Как берем churn rate?

In [117]:
churn_rate_by_industry = {'Энергетика и коммунальные услуги':.11,
                          'ИТ-услуги':.12,
                          'ПО':.14,
                          'Производство промышленного оборудования':.17,
                          'Финансовый сектор услуг':.19,
                          'Профессиональные услуги: бухгалтерские, юридические, инженерные и т.д.':.27,
                          'Телекоммуникации':.31,
                          'Производство товаров':.35,
                          'Транспорт и перевозки':.40,
                          'Производство потребительских товаров':.40,
                          'Оптовая торговля':.56}

Здесь чатГПТ отвечает на вопрос о трекшене и партнерах, чтобы дать пример стартаперу, а потом просим подобную инфу у стартапера.

##### ЧатГПТ отвечает: О трекшене и партнерствах

In [143]:
prompt = '''Напиши пример трекшна и партнерствах в бизнесе по описанию компании.

Термин "трекшн" (англ. "traction") в бизнес-контексте обозначает показатель или показатели, которые демонстрируют рост, прогресс или успешность стартапа или бизнеса. Это может быть количественные данные, которые отражают увеличение клиентской базы, продажи продуктов или услуг, пользовательскую активность, доходы и т.д. Короче говоря, трекшн - это подтверждение того, что бизнес-идея работает на практике и привлекает интерес со стороны пользователей или клиентов.

Когда стартап или компания ищет инвестиции или партнерство, трекшн играет важную роль. Инвесторы и потенциальные партнеры обращают внимание на трекшн, чтобы оценить, насколько успешным и перспективным является бизнес. Увеличение трекшн с течением времени указывает на то, что бизнес-модель работает, а клиенты или пользователи заинтересованы в продукте или услуге.

Примеры показателей трекшн включают в себя:

Количество активных пользователей или клиентов.
Объем продаж продуктов или услуг.
Рост доходов.
Количество скачиваний (для мобильных приложений).
Количество подписчиков или подписок (для подписочных сервисов).
Уровень удержания клиентов (retention rate).
Метрики вовлеченности пользователей (например, время, проведенное на платформе).
Общий смысл трекшн заключается в демонстрации того, что бизнес растет и привлекает внимание аудитории.

Описание компании:
{text}
'''

In [144]:
template = ChatPromptTemplate.from_template(prompt)

In [145]:
chain = LLMChain(
    prompt=template,
    llm=llm, 
    verbose=True
)

In [146]:
description = '''"Компания ООО «РеоКат» работает в области сервиса катализаторов нефтепереработки и систем аминовой очистки газов. Компания специализируется на релизации проектов связанных с:
1. Мобильными комплексами высокоэффективной регенерации наноструктурированных катализаторов гидропроцессов нефтепереработки, включающий блок очистки отходящих газов.
2. Сервисе катализаторных систем: компания  оказывает услуги по выгрузке и загрузке катализаторов и адсорбентов, сульфидированию катализаторов гидроочистки.
3. Компания занимается очисткой аминовых растворов от механических примесей и термостабильных солей.Цеолит представляет собой гранулы, длиной 3-6 мм и диаметром 1-2 мм. При эксплуатации цеолита на установках очистки или осушки в течение нескольких лет на нем накапливаются сера и углерод, которые не могут быть удалены при регенерации в процессе и снижают его емкость. Цеолит выгружают из адсорберов и заменяют на новый. Отработанный цеолит направляется на утилизацию. Срок службы цеолита — от 6 месяцев до 4 лет."
'''

In [147]:
answer = chain(description)



> Entering new LLMChain chain...
Prompt after formatting:
Human: Напиши пример трекшна и партнерствах в бизнесе по описанию компании.

Термин "трекшн" (англ. "traction") в бизнес-контексте обозначает показатель или показатели, которые демонстрируют рост, прогресс или успешность стартапа или бизнеса. Это может быть количественные данные, которые отражают увеличение клиентской базы, продажи продуктов или услуг, пользовательскую активность, доходы и т.д. Короче говоря, трекшн - это подтверждение того, что бизнес-идея работает на практике и привлекает интерес со стороны пользователей или клиентов.

Когда стартап или компания ищет инвестиции или партнерство, трекшн играет важную роль. Инвесторы и потенциальные партнеры обращают внимание на трекшн, чтобы оценить, насколько успешным и перспективным является бизнес. Увеличение трекшн с течением времени указывает на то, что бизнес-модель работает, а клиенты или пользователи заинтересованы в продукте или услуге.

Примеры показателей трекшн вкл

In [148]:
print(answer['text'])

Пример трекшна и партнерства для компании ООО "РеоКат":

Компания ООО "РеоКат" может демонстрировать свой трекшн и привлекать партнеров, предоставляя следующие показатели:

1. Количество активных клиентов: компания может указать, сколько клиентов они обслуживают в настоящее время и как это число растет с течением времени. Например, они могут сообщить, что у них в настоящее время 50 активных клиентов, и это число увеличивается на 10% каждый месяц.

2. Объем продаж продуктов и услуг: компания может предоставить данные о своих ежемесячных продажах и показать, как они растут. Например, они могут сообщить, что их ежемесячный объем продаж составляет 100 000 долларов США, и он увеличивается на 20% каждый квартал.

3. Рост доходов: компания может указать, как их доходы растут с течением времени. Например, они могут сообщить, что их годовой доход составляет 1 миллион долларов США, и он увеличивается на 30% каждый год.

4. Уровень удержания клиентов: компания может указать, как долго их клиенты 

### Команда + борд